## Day 11 - Distances between galaxies!

In [70]:
with open("./example.txt") as f:
    examples_lines = [line.strip() for line in f.readlines()]
with open("./input.txt") as f:
    input_lines = [line.strip() for line in f.readlines()]

In [71]:
examples_lines

['...#......',
 '.......#..',
 '#.........',
 '..........',
 '......#...',
 '.#........',
 '.........#',
 '..........',
 '.......#..',
 '#...#.....']

In [107]:
import copy
def expand_universe(lines: list[str]) -> list[str]:
    """
    Returns an expanded universe.
    """
    new_lines = copy.deepcopy(lines)
    
    # Fill horizontally
    shifts = 0
    for idx, line in enumerate(lines):
        if all([debris == "." for debris in line]):
            new_lines = (
                new_lines[0:idx + shifts] + ["." * len(line)] + new_lines[shifts + idx:]
            )
            shifts += 1

    # Fill vertically
    v_shifts = 0
    for idx in range(len(lines[0])):
        if all([debris_line[idx] == "." for debris_line in lines]):
            # print(new_lines[0], idx, idx + 1 + v_shifts)
            # print(new_lines[0][0: idx + 1 + v_shifts])
            # print(new_lines[0][0: idx + 1 + v_shifts] + "." + new_lines[0][idx + shifts:])
            new_lines = [
                line[0: idx + v_shifts] + "." + line[idx+v_shifts:]
                for line in new_lines
            ]
            v_shifts += 1
    
    
    return new_lines

# expand_universe(examples_lines)

In [110]:
with open("./example_expansion_first_last_row_col.txt") as f:
    example_expansion_lines = [line.strip() for line in f.readlines()]

expand_universe(example_expansion_lines)

['.............',
 '.............',
 '.............',
 '.............',
 '.............',
 '.............',
 '.............',
 '.............',
 '.............',
 '.............',
 '....#######..',
 '.............',
 '.............',
 '.............',
 '.............',
 '.............',
 '.............',
 '.............',
 '.............']

In [73]:
# Might as well number the galaxies as suggested for the sake of pairing!

def number_galaxies(lines: list[str]) -> list[str]:
    """
    Returns fresh set of lines and also the count of the galaxies
    """
    galaxy_number = 1
    new_lines = copy.deepcopy(lines)

    for i in range(len(lines)):
        finding = True
        while finding:
            if finding := (new_lines[i].find("#") >= 0):
                new_lines[i] = new_lines[i].replace("#", str(galaxy_number), 1)
                galaxy_number += 1

    return new_lines

number_galaxies(expand_universe(examples_lines))

['....1........',
 '.........2...',
 '3............',
 '.............',
 '.............',
 '........4....',
 '.5...........',
 '............6',
 '.............',
 '.............',
 '.........7...',
 '8....9.......']

In [74]:
def locations_of_galaxies(lines: list[str]) -> list:
    """
    returns list with (y,x) tuples where
    (0,0) is top left
    """
    locs = []
    for y, line in enumerate(lines):
        for x in range(len(line)):
            if line[x] != ".":
                locs.append((y,x))
    
    return locs

expanded_and_numbered_galaxy = number_galaxies(expand_universe(examples_lines))
galaxy_locs = locations_of_galaxies(expanded_and_numbered_galaxy)
galaxy_locs

[(0, 4), (1, 9), (2, 0), (5, 8), (6, 1), (7, 12), (10, 9), (11, 0), (11, 5)]

In [96]:
import itertools

def part1(lines: list[str]) -> int:
    """
    Sum of the distances
    """
    universe = number_galaxies(expand_universe(lines))
    galaxy_locs = locations_of_galaxies(universe)

    total_distances = 0
    for galaxy_pair in itertools.combinations(galaxy_locs, r=2):
        g1, g2 = galaxy_pair
        y1, x1 = g1
        y2, x2 = g2
        total_distances += abs(x2-x1) + abs(y2-y1)
        # total_distances += (
        #     max(x1,x2) - min(x1, x2) + max(y1, y2) - min(y1, y2)
        # )
    
    return total_distances

assert part1(examples_lines) == 374
part1(input_lines)  # why isn't the answer 73934864  :((((

73934864

In [101]:
with open("./example_expansion_first_last_row_col.txt") as f:
    example_expansion_lines = [line.strip() for line in f.readlines()]

expand_universe(example_expansion_lines)

['............',
 '............',
 '..#.........',
 '..#.........',
 '..#.........',
 '..#.........',
 '..########..',
 '..#.........',
 '..#.........',
 '..#.........',
 '............',
 '............']

In [105]:
# honestly idk what could be going wrong but guess I'll change method???

def part1_2(lines: list[str]) -> int:
    empty_rows = [r for r, row in enumerate(lines) if all(ch == "." for ch in row)]
    empty_cols = [c for c, col in enumerate(zip(*lines)) if all(ch == "." for ch in col)]

    galaxy_points = [(r, c) for r, row in enumerate(lines) for c, ch in enumerate(row) if ch == "#"]

    total = 0
    scale = 2

    for i, (r1, c1) in enumerate(galaxy_points):
        for (r2, c2) in galaxy_points[:i]:
            # so basically manually find the distance but by counting in a loop and double counting the 'empty row' entries!
            for r in range(min(r1, r2), max(r1, r2)):
                total += scale if r in empty_rows else 1
            for c in range(min(c1, c2), max(c1, c2)):
                total += scale if c in empty_cols else 1

    return total            

assert part1_2(examples_lines) == 374
part1_2(input_lines) 
# lol this is a factor of 10 out from the first attempt, 
# no idea what could have other than like the expansion or double counting... but ???
# ohhhhh I wonder if I was expanding the grid like more than once for the same row/col bc of replacement?

# ugh checked and that doesn't seem to be it... stumped.

9742154

**Part 2 - million times expansion instead!**

In [112]:
# so tbf would've had to change strategy here anyway

def part2(lines: list[str], scale = 1000000) -> int:
    empty_rows = [r for r, row in enumerate(lines) if all(ch == "." for ch in row)]
    empty_cols = [c for c, col in enumerate(zip(*lines)) if all(ch == "." for ch in col)]

    galaxy_points = [(r, c) for r, row in enumerate(lines) for c, ch in enumerate(row) if ch == "#"]

    total = 0

    for i, (r1, c1) in enumerate(galaxy_points):
        for (r2, c2) in galaxy_points[:i]:
            # so basically manually find the distance but by counting in a loop and double counting the 'empty row' entries!
            for r in range(min(r1, r2), max(r1, r2)):
                total += scale if r in empty_rows else 1
            for c in range(min(c1, c2), max(c1, c2)):
                total += scale if c in empty_cols else 1

    return total            

assert part2(examples_lines, 10) == 1030
assert part2(examples_lines, 100) == 8410
part2(input_lines)

411142919886

the mystery of the original part1 remains i guess!